# Information Extraction
- 06/28/30 Extract the entity relations related to covid-19
    - Whether there is relationship or not
    - If so, what kind, and how likely
    - Use the openNRE to do so

- Examples: 
    - Risk Factors
        - “The two-way interaction between COVID-19 and diabetes mellitus sets up a vicious cycle wherein COVID-19 leads to worsening of dysglycemia and diabetes mellitus”
            1.	Covid-19 has relationship with diabetes. 
            2.	Covid-19 worsen diabetes
            3.	Does not indicate diabetes is a risk factor; but indicate covid-19 impact diabetes. 
    - Efficacy of therapeutics and interventions
        - i. “Increasing eosinophils may be an indicator of COVID-19 improvement. The COVID-19 patients may benefit from sustained lopinavir use.”
            1.	lopinavir may be efficient for treating covid-19.
            2.	OpenNRE: “eosinophils” has part (P527)  “covid-19”

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from covid.models.relation.extraction import RelationExtractor
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Sample Extraction
- There are two model available: wiki80_bert_softmax or wiki80_cnn_softmax. The first one is better. The model is from the supervised relation extraction: http://opennre.thunlp.ai/#/sent_re. 
- We need to add an additiona NER classifier to classify interesting term relationship (in addition to our defined one)
- Among all the relations, there are several relations that are interesting, e.g., "has part (P527)", "part of (P361)", "said to be the same as", etc.

In [3]:
# initiate the extractor
rextractor = RelationExtractor("../covid/models/paperclassifier/interest.yaml")

In [23]:
# extract
text = 'Increasing eosinophils may be an indicator of COVID-19 improvement. The COVID-19 patients may benefit from sustained lopinavir use.'
relation = rextractor.extract(text, 'eosinophils', 'COVID-19')
print(relation)

relation = rextractor.extract(text, 'WORDNOTINTEXT', 'COVID-19')
print(relation)

('eosinophils', 'covid-19', ('has part', 0.3858766555786133))
None


# Paper relationship extraction
- We can utilize the interest.yml to accelerate the search and relationship extraction
- We will focus on the relationship of covid to a particular keyword

In [64]:
from covid.models.relation.extraction import RelationExtractor

covidre = RelationExtractor(km_path='../covid/models/paperclassifier/interest.yaml')

### Load the paper-classified information

In [47]:
df = pd.read_csv('../data/paperclassifier/classified_merged_covid.csv')
df.head(3)

,sha,title,abstract,publish_time,affiliations,location,text,risk_factor,diagnostic,treatment_and_vaccine,...,imaging_diagnosis,clinical_diagnosis,genetic_diagnosis,treatment_and_vaccine_common_name,treatment,outcome_common_name,clinical_outcome,keywords,affiliations_country,location_country
0,0104f6ceccf92ae8567a0102f89cbb976969a774,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,NaN,NaN,NaN,0,1,0,...,0,0,0,0,0,0,0,fever,NaN,NaN
1,5b68a553a7cbbea13472721cd1ad617d42b40c26,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,NaN,NaN,NaN,1,0,0,...,0,0,0,0,0,0,0,weather,NaN,NaN
2,3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c,Cloaked similarity between HIV-1 and SARS-CoV ...,BACKGROUND: Severe acute respiratory syndrome ...,2003-09-21,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,59ffca26c9d1f881e4e37e7c5af626d72d858dc4,Relationship of SARS-CoV to other pathogenic R...,BACKGROUND: The exact origin of the cause of t...,2003-09-20,Yee Leng Yap ; Xue Wu Zhang ; Antoine Danchin ...,"; ; 75724, Cedex 15 ; Paris ; France",Background\n\nSevere Acute Respiratory Syndrom...,0,1,0,...,0,0,1,0,0,0,0,"rna,dna",France,France
4,c63c4d58d170136b8d3b5a66424b5ac3f73a92d9,Viral Discovery and Sequence Recovery Using DN...,Because of the constant threat posed by emergi...,2003-11-17,David Wang (University of California San Franc...,San Francisco ; California ; United States of ...,Introduction\n\nOver the past two decades tech...,0,1,0,...,0,0,1,0,0,0,0,dna,United States,United States


In [49]:
df.columns

Index(['sha', 'title', 'abstract', 'publish_time', 'affiliations', 'location',
       'text', 'risk_factor', 'diagnostic', 'treatment_and_vaccine', 'outcome',
       'risk_factor_common_name', 'gender', 'age', 'disease_comorbidity',
       'smoking', 'exercise', 'occupation', 'weather',
       'diagnostic_common_name', 'symptom', 'imaging_diagnosis',
       'clinical_diagnosis', 'genetic_diagnosis',
       'treatment_and_vaccine_common_name', 'treatment', 'outcome_common_name',
       'clinical_outcome', 'keywords', 'affiliations_country',
       'location_country'],
      dtype='object')

### Information extraction for each keyword

In [125]:
relations = covidre.extract_all(df)

In [126]:
df['relations'] = relations

### Save the information

In [127]:
df.to_csv('../data/paperclassifier/classified_merged_covid_relation.csv')

# Analyze the relations
- This is more like for setting up examples to better look at the data using declarative tool, i.e., altair

In [4]:
import ast
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# load the data
df_r = pd.read_csv('../data/paperclassifier/classified_merged_covid_relation.csv')
df_r.head(3)

,Unnamed: 0,sha,title,abstract,publish_time,affiliations,location,text,risk_factor,diagnostic,...,clinical_diagnosis,genetic_diagnosis,treatment_and_vaccine_common_name,treatment,outcome_common_name,clinical_outcome,keywords,affiliations_country,location_country,relations
0,0,0104f6ceccf92ae8567a0102f89cbb976969a774,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,NaN,NaN,NaN,0,1,...,0,0,0,0,0,0,fever,NaN,NaN,"[('coronavirus', 'fever', ('part of', 0.206526..."
1,1,5b68a553a7cbbea13472721cd1ad617d42b40c26,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,weather,NaN,NaN,"[('coronavirus', 'weather', ('part of', 0.7285..."
2,2,3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c,Cloaked similarity between HIV-1 and SARS-CoV ...,BACKGROUND: Severe acute respiratory syndrome ...,2003-09-21,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [27]:
import math

# modify the relations columns to make it more readable
# string to list
rs = []
for x in df_r['relations'].tolist():
    if isinstance(x, str):
        rx = ast.literal_eval(x)
        
        # create content string
        ss = []
        for c in rx:
            s = "%s('%s', %.3f)"%(c[1],c[2][0],c[2][1])
            ss.append(s)
        rx = ", ".join(ss)
    else:
        rx = None
    rs.append(rx)
print(len(rs))
rs

9446


["fever('part of', 0.207)",
 "weather('part of', 0.729)",
 None,
 "rna('part of', 0.829), dna('follows', 0.352)",
 "dna('part of', 0.853)",
 None,
 "treatment('has part', 0.488)",
 "rna('has part', 0.734)",
 "rna('part of', 0.499), death('has part', 0.825)",
 "rna('part of', 0.664)",
 None,
 "rna('part of', 0.542)",
 "discharge('has part', 0.599)",
 None,
 None,
 None,
 "fever('said to be the same as', 0.912)",
 None,
 "rna('part of', 0.952)",
 "rna('platform', 0.357)",
 "rna('has part', 0.537)",
 "male('part of', 0.955), sex('part of', 0.955), age('part of', 0.955)",
 "rna('instance of', 0.546)",
 "fever('league', 0.492)",
 "rna('has part', 0.969)",
 "rna('part of', 0.774)",
 "age('has part', 0.612), respiratory tract infection('part of', 0.782), rna('has part', 0.690)",
 None,
 None,
 None,
 None,
 None,
 None,
 "temperature('part of', 0.891), rna('has part', 0.613)",
 "symptom('has part', 0.863), rna('has part', 0.863), treatment('has part', 0.863)",
 "rna('has part', 0.633)",
 None

In [8]:
# ---------- Create a new dataframe to stores explicitly of all the relationship extracted
# string to list
rs = [ast.literal_eval(x) for x in df_r['relations'].dropna().tolist()]

# flatten the arrays
all_rs = []
for r in rs:
    for i in r:
        all_rs.append(i)

# format to pandas
rs = []
for r in all_rs:
    rs.append([r[0], r[1], r[2][0], r[2][1]])
df_new = pd.DataFrame(rs)
df_new.columns = ['keyword1', 'keyword2', 'relation', 'probability']
df_new.head(3)

,keyword1,keyword2,relation,probability
0,coronavirus,fever,part of,0.206526
1,coronavirus,weather,part of,0.728578
2,coronavirus,rna,part of,0.828718


In [172]:
# NOTE: some of the relationships do not make sense. Can ignore them. For example, "sport"
df_new['relation'].value_counts()

has part                                            4094
part of                                             2225
instance of                                          655
follows                                              438
followed by                                          416
main subject                                         382
said to be the same as                               206
league                                               175
characters                                            92
developer                                             33
sport                                                 26
platform                                              19
distributor                                           11
location                                              11
operating system                                       9
performer                                              7
military branch                                        7
sibling                        

### Visualize
- feel free to do any visualization on the data

In [175]:
chart = alt.Chart(df_new).mark_bar().encode(
    alt.X("relation:Q", bin=True),
    y='count()',
)
chart

alt.Chart(...)

In [176]:
df['relations'].tolist()

[[('coronavirus', 'fever', ('part of', 0.20652639865875244))],
 [('coronavirus', 'weather', ('part of', 0.7285776138305664))],
 None,
 [('coronavirus', 'rna', ('part of', 0.8287178874015808)),
  ('coronavirus', 'dna', ('follows', 0.3523031771183014))],
 [('coronavirus', 'dna', ('part of', 0.8533002734184265))],
 None,
 [('coronavirus', 'treatment', ('has part', 0.4881404638290405))],
 [('coronavirus', 'rna', ('has part', 0.7344115376472473))],
 [('coronavirus', 'rna', ('part of', 0.4985741078853607)),
  ('coronavirus', 'death', ('has part', 0.8249072432518005))],
 [('coronavirus', 'rna', ('part of', 0.6635359525680542))],
 None,
 [('coronavirus', 'rna', ('part of', 0.5419868230819702))],
 [('coronavirus', 'discharge', ('has part', 0.5988366603851318))],
 None,
 None,
 None,
 [('coronavirus', 'fever', ('said to be the same as', 0.9116886854171753))],
 None,
 [('coronavirus', 'rna', ('part of', 0.9524491429328918))],
 [('coronavirus', 'rna', ('platform', 0.3569752275943756))],
 [('corona